# 以下是神经网络方法。

In [1]:
import os
from datetime import datetime

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import SGD, Adagrad, Adam
from torchvision import datasets, models, transforms
from torchvision.models import (ResNet18_Weights, ResNet50_Weights,
                                VGG11_Weights, resnet18, resnet50, vgg11)
from tqdm import tqdm

from dataset import MyDataset
from my_mlp import MyMLP
from utils import label_int2str, label_str2int, save_config


device = "cuda"
assert torch.cuda.is_available(), "你TM连CUDA都没你跑个JB"

CUDA availability: True


## 运行设置：

In [2]:
# Run name
run_name = datetime.now().strftime("%Y-%m-%d %H.%M.%S")

# Network architecture
# mlp, resnet18, resnet50, vgg11
model_name = "resnet18"
model_is_pretrained = False
num_classes = 6

# Optimizer
# SGD, Adagrad, Adam
optimizer_name = "Adam"
lr = 1e-3

# Data augmentation
augmentation = "none"

# Regularization
regularization = "none"

save_config(run_name, model_name, model_is_pretrained, num_classes, optimizer_name, lr, augmentation, regularization)


    # Run name
    run_name = "2023-05-14 21.22.16"

    # Network architecture
    # mlp, resnet18, resnet50, vgg11
    model_name = "resnet18"
    model_is_pretrained = False
    num_classes = 6

    # Optimizer
    # SGD, Adagrad, Adam
    optimizer_name = "Adam"
    lr = 0.001000

    # Data augmentation
    augmentation = "none"

    # Regularization
    regularization = "none"
    


## 模型网络架构：

(Reference from pytorch.org)

| Weight |Acc@1 |Acc@5 |Params |GFLOPS|
| ---- | ---- | ---- | ---- | ---- |
| ResNet18_Weights.IMAGENET1K_V1 | 69.758 | 89.078 | 11.7M | 1.81 |
| ResNet50_Weights.IMAGENET1K_V2 | 80.858 | 95.434 | 25.6M | 4.09 |
| VGG11_Weights.IMAGENET1K_V1 | 69.02 | 88.628 | 132.9M | 7.61 |

In [3]:
if model_name=="MLP":
    model = MyMLP(150*150, num_classes)
    input_size = (150, 150)
elif model_name=="resnet18":
    if model_is_pretrained:
        model = resnet18(weights=ResNet18_Weights.DEFAULT)
    else:
        model = resnet18(weights=None)
    input_size = (224, 224)
    model.fc = torch.nn.Linear(512, num_classes)
elif model_name=="resnet50":
    if model_is_pretrained:
        model = resnet50(weights=ResNet50_Weights.DEFAULT)
    else:
        model = resnet50(weights=None)
    input_size = (224, 224)
    model.fc = torch.nn.Linear(512, num_classes)
elif model_name=="vgg11":
    if model_is_pretrained:
        model = vgg11(weights=VGG11_Weights.DEFAULT)
    else:
        model = vgg11(weights=None)
    input_size = (224, 224)
    model.classifier[6] = torch.nn.Linear(4096,num_classes)

model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 优化器：

In [4]:
if optimizer_name=="SGD":
    optimizer = SGD(model.parameters(), lr=lr)
elif optimizer_name=="Adagrad":
    optimizer = Adagrad(model.parameters(), lr=lr)
elif optimizer_name=="Adam":
    optimizer = Adam(model.parameters(), lr=lr)

## 数据增强与预处理：

In [5]:
if augmentation=="none":
    transform = transforms.Compose([
        transforms.Resize(input_size), 
        transforms.ToTensor()
    ])
elif augmentation=="flip":
    transform = transforms.Compose([
        transforms.Resize(input_size), 
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor()
    ])
elif augmentation=="crop":
    transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size), 
        transforms.ToTensor()
    ])
elif augmentation=="norm":    
    transform = transforms.Compose([
        transforms.Resize(input_size), 
        transforms.Normalize(mean=(0, 0, 0), std=(1, 1, 1)), 
        transforms.ToTensor()
    ])
elif augmentation=="all":
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), 
        transforms.RandomResizedCrop(input_size), 
        transforms.Normalize(mean=(0, 0, 0), std=(1, 1, 1)), 
        transforms.ToTensor()
    ])

dataset = datasets.ImageFolder("./dataset/seg_train/", transform)
testset = datasets.ImageFolder("./dataset/seg_test/") # Do not transform input images in testing. 
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8)
testloader = DataLoader(testset, batch_size=32, shuffle=True, num_workers=8)

## 训练过程：

In [6]:
def train_one_epoch(model: torch.nn.Module, dataloader: DataLoader, transform: transforms.Compose, 
                    optimizer: torch.optim.Optimizer, current_epoch: int):
    correct_predictions = 0
    epoch_instance_count = 0
    epoch_loss = 0
    for inputs, labels in tqdm(dataloader, desc="Training epoch %d"%current_epoch):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()*inputs.size(0)
        _, predictions = torch.max(outputs, 1)
        correct_predictions += torch.sum(predictions==labels.data)
        epoch_instance_count += inputs.size(0)
    epoch_accuracy = correct_predictions/epoch_instance_count
    epoch_loss /= epoch_instance_count
    if current_epoch%1==0:
        print("Epoch %d (%d instances), loss = %f, accuracy = %f"%(current_epoch, epoch_instance_count, epoch_loss, epoch_accuracy))
    if current_epoch%10==0:
        torch.save(model, "./outputs/runs/%s/model_epoch_%d"%(run_name, current_epoch))
    return epoch_accuracy, epoch_loss

def test(model: torch.nn.Module, dataloader: DataLoader):
    correct_predictions = 0
    epoch_instance_count = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            correct_predictions += torch.sum(predictions==labels.data)
            epoch_instance_count += inputs.size(0)
    epoch_accuracy = correct_predictions/epoch_instance_count
    print("Testing (%d instances), accuracy = %f"%(epoch_instance_count, epoch_accuracy))

epoch_accuracies = []
epoch_losses = []
model.train()
for epoch in range(1, 30):
    train_one_epoch(model, dataloader, transform, optimizer, epoch)
test(model, testloader)

Training epoch 1: 100%|██████████| 439/439 [00:28<00:00, 15.31it/s]


Epoch 1 (14034 instances), loss = 0.927014, accuracy = 0.639091


Training epoch 2: 100%|██████████| 439/439 [00:25<00:00, 17.13it/s]


Epoch 2 (14034 instances), loss = 0.640746, accuracy = 0.763075


Training epoch 3: 100%|██████████| 439/439 [00:25<00:00, 17.01it/s]


...

Training epoch 7: 100%|██████████| 439/439 [00:25<00:00, 16.90it/s]


Epoch 7 (14034 instances), loss = 0.394481, accuracy = 0.857204


Training epoch 8: 100%|██████████| 439/439 [00:25<00:00, 16.90it/s]


Epoch 8 (14034 instances), loss = 0.351716, accuracy = 0.876728


Training epoch 9:  51%|█████▏    | 226/439 [00:16<00:09, 22.51it/s]